### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 32.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 16.5 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.199.0-py2.py3-none-any.whl size=1374440 sha256=b1627ae7d62fd0ec24bbd3a5dbbe

-  可以把下面cell中的en全部切换成zh即可部署中文版

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-zh-model")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-large-zh-v1.5"
commit_hash = "00f8ffc4928a685117583e2a38af8ebb65dcec2c"

In [3]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

'bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c'

### 2. 把模型拷贝到S3为后续部署做准备

In [4]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
s3_model_prefix = "LLM-RAG/workshop/bge-zh-model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/bge_zh_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/bge_zh_deploy_code
model_snapshot_path: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c


In [11]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c/.gitattributes to s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/.gitattributes
upload: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c/1_Pooling/config.json to s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/1_Pooling/config.json
upload: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c/README.md to s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/README.md
upload: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c/config_sentence_transformers.json to s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/config_sentence_transformers.json
upload: bge-zh-model/models--BAAI--bge-large-zh-v1.5/snapshots/00f8ffc4928a685117583e2a38af8ebb65dcec2c/sentence_bert_config.json to s3://

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [12]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.25.0-deepspeed0.11.0-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-deepspeed0.11.0-cu118


In [13]:
!mkdir -p bge_zh_deploy_code

In [14]:
%%writefile bge_zh_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from FlagEmbedding import FlagModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    model =  FlagModel(model_location)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    is_query = data["is_query"]
    instruction = data["instruction"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"is_query: {is_query}")
    logging.info(f"instruction: {instruction}")
    
    if is_query and instruction:
        input_sentences = [ instruction + sent for sent in input_sentences ]
        
    sentence_embeddings =  model.encode(input_sentences)
        
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Overwriting bge_zh_deploy_code/model.py


In [15]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [16]:
%%writefile bge_zh_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge-zh-model/

Writing bge_zh_deploy_code/serving.properties


In [17]:
%%writefile bge_zh_deploy_code/requirements.txt
transformers>=4.28.1
FlagEmbedding

Writing bge_zh_deploy_code/requirements.txt


In [18]:
!rm s2e_model.tar.gz
!cd bge_zh_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz bge_zh_deploy_code

rm: cannot remove ‘s2e_model.tar.gz’: No such file or directory
bge_zh_deploy_code/
bge_zh_deploy_code/serving.properties
bge_zh_deploy_code/model.py
bge_zh_deploy_code/requirements.txt


In [19]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-981186762119/LLM-RAG/workshop/bge_zh_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [20]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-zh-15") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-zh-15-2023-11-30-07-36-39-352
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-deepspeed0.11.0-cu118
Created Model: arn:aws:sagemaker:us-west-2:981186762119:model/bge-zh-15-2023-11-30-07-36-39-352


In [21]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:981186762119:endpoint-config/bge-zh-15-2023-11-30-07-36-39-352-config',
 'ResponseMetadata': {'RequestId': '985d55bf-8b8e-468c-89d5-36569f8795c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '985d55bf-8b8e-468c-89d5-36569f8795c1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Thu, 30 Nov 2023 07:36:44 GMT'},
  'RetryAttempts': 0}}

In [22]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:981186762119:endpoint/bge-zh-15-2023-11-30-07-36-39-352-endpoint


In [28]:
import time

endpoint_name = "bge-zh-15-2023-11-30-07-36-39-352-endpoint"
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:981186762119:endpoint/bge-zh-15-2023-11-30-07-36-39-352-endpoint
Status: InService


### 5. 模型测试

In [29]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "is_query": True,
                "instruction" :  "Represent this sentence for searching relevant passages:"
            }
        ),
        ContentType="application/json",
    )
    # 中文instruction => 为这个句子生成表示以用于检索相关文章：
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [30]:
prompts1 = ["你好啊，大聪明"]
endpoint_name= 'bge-zh-15-2023-11-30-07-36-39-352-endpoint'
emb = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)
print(emb[0][:10])

[0.0089569091796875, -0.01233673095703125, -0.026702880859375, -0.02386474609375, 0.04791259765625, 0.0023632049560546875, -0.0562744140625, -0.00807952880859375, 0.006664276123046875, -0.029998779296875]


In [ ]:
!aws sagemaker delete-endpoint --endpoint-name bge-large-en-2023-08-16-09-58-49-900-endpoint

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name bge-large-en-2023-08-16-09-58-49-900-config

In [ ]:
!aws sagemaker delete-model --model-name bge-large-en-2023-08-16-09-58-49-900